In [26]:
import pandas as pd, numpy as np
import os

db = 'C:\\Datasets\\thesis.db'
conn = sqlite3.connect(db)
c = conn.cursor()

directory = ['factset']
tables = {}

for file in os.listdir('//'.join(directory)):
    if '.xls' in file:
        file_path = '\\'.join(directory+[file])
        tables[file[:file.find('.')]] = pd.read_excel(file_path, skiprows=7)
#print(list(tables.keys()))

for table_name in tables.keys():
    print(table_name)
    if 'Key-Developments' in table_name:
        table = tables[table_name]
        drop_phrase = ' dropped from '
        add_phrase = ' added to '
        table['Index'] = table['Key Development Headline'].apply(lambda headline: 
                                                                 headline[headline.find(drop_phrase)-len(headline)+len(drop_phrase):] 
                                                                 if drop_phrase in headline 
                                                                 else headline[headline.find(add_phrase)-len(headline)+len(add_phrase):])
        table['Action'] = table['Action'].apply(lambda action: 'Delete' if 'Drop' in action else 'Add')
        table['Ticker_Pair'] = table['Company Name'].apply(lambda name: name[name.rfind('('):][1:-1])
        table['Ticker'] = table['Ticker_Pair'].apply(lambda ticker_pair: ticker_pair[:ticker_pair.find(':')])
        table['Exchange'] = table['Ticker_Pair'].apply(lambda ticker_pair: ticker_pair[ticker_pair.find(':')+1:])
        table['Name'] = table['Company Name'].apply(lambda name: name[:name.rfind('(')])
        table = table[['Key Developments By Date','Action','Index','Ticker','Exchange','Name']]
        table.columns = ['ticker','action','index','exchange','ticker','name']
        tables[table_name]=table
        table.to_sql('factset_index_changes', conn, if_exists='replace', index=False)
    elif 'Holdings' in table_name:
        table = tables[table_name]
        
        table['Ticker_Pair'] = table['Ticker']
        table['Exchange'] = table['Ticker_Pair'].apply(lambda ticker_pair: ticker_pair[:ticker_pair.find(':')])
        table['Ticker'] = table['Ticker_Pair'].apply(lambda ticker_pair: ticker_pair[ticker_pair.find(':')+1:])
        table = table[['Ticker','Exchange','Index Constituents']]
        
        split_indices = []
        for i, row in table.iterrows():
            indices = row['Index Constituents'].split('; ')
            if len(indices)>1:
                for index in indices:
                    working_row = row.copy(deep=True)
                    working_row['Index Constituents'] = index
                    split_indices.append(pd.DataFrame(working_row).T)
            else:
                split_indices.append(pd.DataFrame(row).T)
        table = pd.concat(split_indices)
        table = table.reset_index().drop(['index'],axis=1)
        #table.columns = ['ticker','exchange','index']
        tables[table_name]=table
        table.to_sql('index_holdings_2019', conn, if_exists='replace', index=False)
    elif 'Sector' in table_name:
        table = tables[table_name]
        table['Ticker_Pair'] = table['Ticker']
        table['Exchange'] = table['Ticker_Pair'].apply(lambda ticker_pair: ticker_pair[:ticker_pair.find(':')]
                                                      if type(ticker_pair)==type('  ') else ticker_pair)
        table['Ticker'] = table['Ticker_Pair'].apply(lambda ticker_pair: ticker_pair[ticker_pair.find(':')+1:]
                                                      if type(ticker_pair)==type('  ') else ticker_pair)
        table.columns = ['ticker','share_type','name','industries','sector','comps','supls','industry','corp_type','exchange_name','ticker_pair','exchange']
        tables[table_name]=table
        table.to_sql('sectors', conn, if_exists='replace', index=False)
    else:
        print(' > Unhandled table',table_name)

KeyboardInterrupt: 

In [23]:
table_clean = pd.read_excel('factset\\All-Canadian-Index_Ratios_1.xls', skiprows=7)
table = table_clean.copy(deep=True)

table = table.replace('-',np.nan)
table = table.replace('NM',np.nan)
columns = []
for column in table.columns:
    newcol = column
    if '%' in column:
        table[column]=table[column]/100
        column = column.replace('%','')
    if '[CQ' in column:
        date = column[column.find('[')+1:column.find(']')].split(' ')
        year=int(date[1])
        quarter=int(date[0][2:])
        
        factor = column[:column.find('[')]
        factors = factor.split(' ')
        factors = [for ]
        
        newcol = '_'.join([factor, str(year), 'Q', str(quarter)])
    newcol = newcol.replace(' ','')
    columns.append(newcol)
table.columns = columns

In [ ]:
tables['All-Canadian-Index_Ratios_1']

In [ ]:
table.columns